In [24]:
import numpy as np
import scipy.special as ss 

In [25]:
class VBA(object):
    def __init__(self, K, Y):
        self.K = K
        self.Y = Y
        self.N = Y.shape[0]
        self.B = np.zeros((self.K, self.K))
        self.alpha = np.zeros((1, self.K))
        
        self.phi_pq = {}
        self.phi_qp = {}
        for i in xrange(self.N-1):
            for j in xrange(i+1, self.N):
                self.phi_pq[(i,j)] = np.zeros((1, self.K))
                self.phi_qp[(i,j)] = np.zeros((1, self.K))
        
        self.gamma = np.zeros((self.N, self.N))
        self.rho = 0
        
    def indOrder(i, j):
        if i > j:
            return (i,j)
        else:
            return (j,i)

    def update_gamma(self):
        for p in xrange(self.N):
            for k in xrange(self.N):
                self.gamma[p,k] = self.alpha[k]
                for q in xrange(self.N):
                    if q != p:
                        self.gamma[p,k] += self.phi_pq[indOrder(p,q)][k] + self.phi_qp[indOrder(p,q)][k]
                        
    def update_B(self):
        for g in xrange(self.K):
            for h in xrange(self.K):
                numerator, denominator = 0, 0
                for p in xrange(self.N):
                    for q in xrange(self.N):
                        if p != q:
                            numerator += self.Y[p,q] * self.phi_pq[indOrder(p,q)][g] * self.phi_qp[indOrder(p,q)][h]
                            denominator += (1 - self.rho) * self.phi_pq[indOrder(p,q)][g] * self.phi_qp[indOrder(p,q)][h]
                self.B[g,h] = numerator / denominator
                
    def update_rho(self):
        numerator, denominator = 0, 0
        for p in xrange(self.N):
            for q in xrange(self.N):
                if p != q:
                    summ = 0
                    for g in xrange(self.K):
                        for h in xrange(self.K):
                            summ += self.phi_pq[indOrder(p,q)][g] * self.phi_qp[indOrder(p,q)][h]
                            denominator += self.phi_pq[indOrder(p,q)][g] * self.phi_qp[indOrder(p,q)][h]
                numerator += (1 - self.Y[p,q]) * summ
        self.rho = numerator / denominator
        
    def update_phi(self):
        phi_pq = {}
        phi_qp = {}
        for i in xrange(self.N-1):
            for j in xrange(i+1, self.N):
                phi_pq[(i,j)] = np.zeros((1, self.K))
                phi_qp[(i,j)] = np.zeros((1, self.K))
                
        for p in xrange(self.N-1):
            for q in xrange(i+1, self.N):
                for g in xrange(self.K):
                    product = 1
                    for h in xrange(self.K):
                        product *= (self.B[g,h]**(self.Y[p,q]) * (1 - self.B[g,h])**(1 - self.Y[p,q]))**(self.phi_qp[indOrder(p,q)][h])
                    phi_pq[indOrder(p,q)][g] = product * np.exp(ss.psi(self.alpha[g]) - ss.psi(np.sum(self.alpha)))
                
        for p in xrange(self.N-1):
            for q in xrange(i+1, self.N):
                for h in xrange(self.K):
                    product = 1
                    for g in xrange(self.K):
                        product *= (self.B[g,h]**(self.Y[p,q]) * (1 - self.B[g,h])**(1 - self.Y[p,q]))**(self.phi_pq[indOrder(p,q)][g])
                    phi_qp[indOrder(p,q)][h] = product * np.exp(ss.psi(self.alpha[h]) - ss.psi(np.sum(self.alpha)))
        
        for i in xrange(self.N-1):
            for j in xrange(i+1, self.N):
                phi_pq[(i,j)] = phi_pq[(i,j)] / np.sum(phi_pq[(i,j)])
                phi_qp[(i,j)] = phi_qp[(i,j)] / np.sum(phi_qp[(i,j)])
                
        self.phi_pq = phi_pq
        self.phi_qp = phi_qp
        
        
        
    def print_params(self):
        print self.gamma

In [26]:
Y = np.array([[0, 0, 1],
             [1, 0, 0],
             [0, 1, 1]])
a = VBA(2, Y)
a.print_params()

[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
